In [1]:
import numpy as np
import sympy as sp
from sympy.interactive.printing import init_printing
from IPython.display import display, Math

In [2]:
init_printing(use_unicode=False, wrap_line=False)

# Wedderburn 

In [3]:
C = np.random.randint(size = (6,4), high = 1000, low = 1)
sp.Matrix(C)

[445  890  726  671]
[                  ]
[88   834  459  699]
[                  ]
[548  152  673  982]
[                  ]
[643  791  478  437]
[                  ]
[110  420  259  132]
[                  ]
[329  832  747  662]

In [4]:
def wedd(A):
    B = A
    r = np.linalg.matrix_rank(A)
    (m,n) = A.shape
    i1 = np.identity(max(m,n))[np.ix_(list(range(m)),list(range(n)))]
    i2 = i1.T
    k = 0
    ans = []
    ans.append(B)
    ind = np.nonzero(B)
    non = (ind[0][0],ind[1][0])
    s = []
    while(abs(B[non]) > 1e-5):
        ei = i2[:,[non[1]]]
        ej = i1[:,[non[0]]]
        w = np.matmul(ej.T,np.matmul(B,ei))
        eij = np.matmul(ei,ej.T)
        s.append((np.matmul(B,np.matmul(eij,B))/w))
        B = B - s[k]
        B[abs(B)<1e-8] = 0
        ans.append(B)
        ind = np.nonzero(B)
        try:
            non = (ind[0][0],ind[1][0])
        except IndexError:
            non = (0,0)
        k += 1
    return(ans,s)

In [5]:
a = wedd(C)

### The Wedderburn Matrices

In [6]:
for i in range(len(a[0])):
    t = sp.latex(sp.Matrix(a[0][i]))
    r = r'{}'.format(t)
    display(Math(r))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [9]:
np.linalg.matrix_rank(a[0][2])

2

### Matrix A as a sum of rank one Matrices

In [10]:
for i in range(len(a[1])):
    t = sp.latex(sp.Matrix(a[1][i]))
    r = r'{}'.format(t)
    display(Math(r))
(m,n) = a[1][0].shape
p = np.zeros((m,n))
for i in range(len(a[1])):
    p = p + a[1][i]
sp.Matrix(p)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

[445.0  890.0  726.0  671.0]
[                          ]
[88.0   834.0  459.0  699.0]
[                          ]
[548.0  152.0  673.0  982.0]
[                          ]
[643.0  791.0  478.0  437.0]
[                          ]
[110.0  420.0  259.0  132.0]
[                          ]
[329.0  832.0  747.0  662.0]

In [18]:
np.linalg.matrix_rank(a[1][0])

1

# Biconjugation

In [9]:
def prin(A):
    (m,n) = A.shape
    l = []
    flag = 0
    for i in range(min(m,n)):
        l.append(i)
        if np.linalg.det(A[np.ix_(l,l)]) == 0:
            return(0)
    return(1)

In [50]:
def bic(A, s):
    m = A.shape[0]
    n = A.shape[1]
    r = np.linalg.matrix_rank(A)
    u = []
    v = []
    w = []
    if s == 'trapldu':
        X = np.identity(n)[np.ix_(list(range(n)),list(range(r)))]
        Y = np.identity(m)[np.ix_(list(range(m)),list(range(r)))]
    elif s == 'qr':
        X = np.identity(n)
        Y = A
    elif s == 'cholesky':
        X = np.identity(m)
        Y = np.identity(m)
    else:
        print('Invalid Choice')
        return()
    u.append(X[:,[0]])
    v.append(Y[:,[0]])
    for k in range(1,r):
        sum1 = np.zeros((n,1))
        sum2 = np.zeros((m,1))
        for j in range(k):
            sum1 = sum1 + ((float(np.matmul(np.matmul(v[j].T,A),X[:,[k]]))/float(np.matmul(np.matmul(v[j].T,A),u[j])))*u[j])
            sum2 = sum2 + ((float(np.matmul(np.matmul(Y[:,[k]].T,A),u[j]))/float(np.matmul(np.matmul(v[j].T,A),u[j])))*v[j])
        u.append(X[:,[k]] - sum1)
        v.append(Y[:,[k]] - sum2)
    for k in range(r):
        w.append(float(np.matmul(np.matmul(v[k].T,A),u[k])))
    U = np.concatenate(u, axis = 1)
    V = np.concatenate(v, axis = 1)
    W = np.diag(w)
    if s == 'cholesky':
        W1 = np.sqrt(W)
        R = np.matmul(W1,np.linalg.inv(U))
        return(R)
    elif s == 'qr':
        W1 = np.sqrt(W)
        R = np.matmul(W1,np.linalg.inv(U))
        Q = np.matmul(V,np.linalg.inv(W1))
        return(Q,R)
    elif s == 'trapldu':
        L = np.matmul(A,U)
        D = np.linalg.inv(W)
        U = np.matmul(V.T,A)
        return(L,D,U)
    else:
        print('Invalid Choice')
        return()

# QR Decomposition

In [30]:
C = np.random.randint(size = (4,3), high = 1000, low = 1)
sp.Matrix(C)

[27   132  517]
[             ]
[822  476  196]
[             ]
[683  384  657]
[             ]
[369  190  29 ]

In [31]:
R = bic(C, 'qr')

In [32]:
t = sp.latex(sp.Eq(sp.Matrix(C), sp.MatMul(sp.Matrix(R[0]), sp.Matrix(R[1]))))
r = r'{}'.format(t)
display(Math(r))

<IPython.core.display.Math object>

In [33]:
sp.Matrix(np.matmul(R[0],R[1]))

[27.0   132.0  517.0]
[                   ]
[822.0  476.0  196.0]
[                   ]
[683.0  384.0  657.0]
[                   ]
[369.0  190.0  29.0 ]

In [34]:
print('Q = ')
sp.Matrix(R[0])

Q = 


[0.0238735935730463   0.98262745175905    0.0439678603887169]
[                                                           ]
[0.726818293223853   0.0728454000568019   -0.58410406261796 ]
[                                                           ]
[0.603913496681133   -0.0364181294056424  0.792635423006578 ]
[                                                           ]
[0.326272445498299   -0.166764980108757   -0.169169611577304]

In [122]:
print('R = ')
sp.Matrix(R[1])

R = 


[929.693497879812  737.449494444708  750.223596906524]
[                                                    ]
[      0.0         577.580507932222  381.787447312315]
[                                                    ]
[      0.0               0.0         563.340837965245]

# Cholesky Factorisation

In [123]:
A = np.random.randint(low = 1,high = 100, size = (3,3))
C = np.dot(A,A.T)

In [124]:
sp.Matrix(C)

[2821  1516  1484]
[                ]
[1516  1046  2079]
[                ]
[1484  2079  7881]

In [125]:
R = bic(C, 'cholesky')

In [126]:
t = sp.latex(sp.Eq(sp.Matrix(C), sp.MatMul(sp.Matrix(R.T), sp.Matrix(R))))
r = r'{}'.format(t)
display(Math(r))

<IPython.core.display.Math object>

In [127]:
sp.Matrix(np.matmul(R.T,R))

[2821.0  1516.0  1484.0]
[                      ]
[1516.0  1046.0  2079.0]
[                      ]
[1484.0  2079.0  7881.0]

In [128]:
print('R = ')
sp.Matrix(R)

R = 


[53.113086899558  28.5428712299645   27.940383182894 ]
[                                                    ]
[      0.0        15.2086982332369  84.2610735673755 ]
[                                                    ]
[      0.0              0.0         0.637549108993737]

# LDU Facatorisation

In [44]:
C = np.random.randint(size = (7,3), high = 1000, low = 1)
sp.Matrix(C)

[492  406  19 ]
[             ]
[86   649  258]
[             ]
[881  650  634]
[             ]
[18   482  533]
[             ]
[568  670  175]
[             ]
[219  310  116]
[             ]
[633  640  277]

In [45]:
R = bic(C,'trapldu')

In [46]:
a0 = R[0]
a2 = R[2]
a1 = R[1]

In [47]:
a0[abs(a0)<1e-10] = 0
a2[abs(a2)<1e-10] = 0
a1[abs(a1)<1e-10] = 0

In [48]:
t = sp.latex(sp.Eq(sp.Matrix(C), sp.MatMul(sp.MatMul(sp.Matrix(a0), sp.Matrix(a1), sp.Matrix(a2)))))
r = r'{}'.format(t)
display(Math(r))

<IPython.core.display.Math object>

In [49]:
sp.Matrix(np.matmul(np.matmul(R[0],R[1]),R[2]))

[492.0  406.0  19.0 ]
[                   ]
[86.0   649.0  258.0]
[                   ]
[881.0  650.0  634.0]
[                   ]
[18.0   482.0  533.0]
[                   ]
[568.0  670.0  175.0]
[                   ]
[219.0  310.0  116.0]
[                   ]
[633.0  640.0  277.0]

In [182]:
print('L = ')
sp.Matrix(a0)

L = 


[1.0   0.0          0.0       ]
[                             ]
[2.0  -7.0          0.0       ]
[                             ]
[3.0  -11.0  0.428571428571431]

In [183]:
print('D =')
sp.Matrix(a1)

D =


[1.0         0.0                0.0       ]
[                                         ]
[0.0  -0.142857142857143        0.0       ]
[                                         ]
[0.0         0.0          2.33333333333334]

In [184]:
print('U =')
sp.Matrix(a2)

U =


[1.0  5.0          8.0         1.0  5.0 ]
[                                       ]
[0.0  -7.0        -13.0        0.0  -7.0]
[                                       ]
[0.0  0.0   0.428571428571428  0.0  0.0 ]

In [ ]:
C = np.matrix([[1,2,3],[5,3,4],[8,3,4],[1,2,3],[5,3,4]])
sp.Matrix(C)